<font color="grey">
    
# classify emotions in English text data

## predicts Ekman's 6 basic emotions, plus a neutral class:
anger 🤬
disgust 🤢
fear 😨
joy 😀
neutral 😐
sadness 😭
surprise 😲

More about the model: https://huggingface.co/j-hartmann/emotion-english-distilroberta-base

</font>

In [1]:
# Load the data
import pandas as pd
from tqdm import tqdm
books=pd.read_csv(r"F:\learning\LLM\step3_books_with_categories.csv")

In [2]:
# Load the model
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)

C:\Users\li\anaconda3\envs\LLM\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Device set to use cuda:0


In [3]:
# Try the model with a simple example
out=classifier("I love this!")[0]
print(pd.DataFrame(out))

      label     score
0       joy  0.977169
1  surprise  0.008529
2   neutral  0.005765
3     anger  0.004420
4   sadness  0.002092
5   disgust  0.001612
6      fear  0.000414


In [4]:
# Try one book in our book database
print(f"The world lenth: {len(books['description'][0].split())}")
out=classifier(books['description'][0])[0]
print("The predicted emotions:")
print(pd.DataFrame(out))

The world lenth: 199
The predicted emotions:
      label     score
0      fear  0.654841
1   neutral  0.169852
2   sadness  0.116409
3  surprise  0.020701
4   disgust  0.019101
5       joy  0.015161
6     anger  0.003935


In [11]:
# Try another book with very long text.
o1=books['description'].apply(lambda x: len(x.split()))
book_id=o1.idxmax()
print(f"The world lenth: {len((books['description'][book_id]).split())}")
out=classifier(books['description'][book_id])[0]
print("The predicted emotions:")
print(pd.DataFrame(out))

Token indices sequence length is longer than the specified maximum sequence length for this model (1177 > 512). Running this sequence through the model will result in indexing errors


The world lenth: 920


RuntimeError: The expanded size of the tensor (1177) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 1177].  Tensor sizes: [1, 514]

<font color="grey">
    
- We encounter an error because the book's text description exceeds the model's processing limits.

We will split each book's text description into individual sentences and apply the model to each sentences. The aggregated emotion predictions across all sentences will be used as the book's overall emotion.

</font>

In [ ]:
def sentiment_each_book(text_des):
    out=classifier(text_des.split("."))#sentense by sentense
    df=pd.DataFrame(out[0]).set_index('label')
    for i in range(1,len(out)):
        dfi=pd.DataFrame(out[i]).set_index('label')
        df=pd.concat([df,dfi],axis=1)
    return df.max(axis=1)

def sentiment_all_books(books):
    df=sentiment_each_book(books['description'][0])
    df.name=books['isbn13'][0]
    for i in tqdm(range(1,len(books))):
        dfi=sentiment_each_book(books['description'][i])
        dfi.name=books['isbn13'][i]
        df=pd.concat([df,dfi],axis=1)
    df=df.T.reset_index().rename(columns={'index':'isbn13'})
    return df
books_with_sentiment=sentiment_all_books(books)    
books_emotion=pd.merge(books,books_with_sentiment,on='isbn13')
books_emotion.to_csv(r"F:\learning\LLM\step4_books_with_emotions.csv", index = False)

<font color="grey">
    
## The whole code
</font>

In [ ]:
import pandas as pd
from tqdm import tqdm
books=pd.read_csv(r"F:\learning\LLM\step3_books_with_categories.csv")

from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)
out=classifier("I love this!")[0]
print(pd.DataFrame(out))

def sentiment_each_book(text_des):
    out=classifier(text_des.split("."))#sentense by sentense
    df=pd.DataFrame(out[0]).set_index('label')
    for i in range(1,len(out)):
        dfi=pd.DataFrame(out[i]).set_index('label')
        df=pd.concat([df,dfi],axis=1)
    return df.max(axis=1)

def sentiment_all_books(books):
    df=sentiment_each_book(books['description'][0])
    df.name=books['isbn13'][0]
    for i in tqdm(range(1,len(books))):
        dfi=sentiment_each_book(books['description'][i])
        dfi.name=books['isbn13'][i]
        df=pd.concat([df,dfi],axis=1)
    df=df.T.reset_index().rename(columns={'index':'isbn13'})
    return df
    
books_with_sentiment=sentiment_all_books(books)    
books_emotion=pd.merge(books,books_with_sentiment,on='isbn13')
books_emotion.to_csv(r"F:\learning\LLM\step4_books_with_emotions.csv", index = False)